In [15]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.datasets import load_boston
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import linear_model
from scipy import stats
import numpy as np


class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    # def __init__(self, *args, **kwargs):
    #     if not "fit_intercept" in kwargs:
    #         kwargs['fit_intercept'] = False
    #     super(LinearRegression, self).__init__(*args, **kwargs)

    def __init__(self):
        super(LinearRegression, self).__init__(fit_intercept=False)

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(
                np.diagonal(
                    sse * np.linalg.inv(
                        np.dot(X.T, X)
                    )
                )
            )
        ])

        self.X = X
        self.y = y
        self.se = se
        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self

    def summary(self):
        columns = ['Estimate', 'Std. Error', 't statistic', 'P value']
        index = self.X.columns
        data = np.array([
            self.coef_,
            self.se[0],
            self.t[0],
            self.p[0]
        ]).T
        
        return pd.DataFrame(data=data, index=index, columns=columns)

In [2]:
# load data
boston = load_boston()
boston_X = boston['data']
boston_y = boston['target']

# transform into dataframe
boston_X_df = pd.DataFrame(data=boston_X, columns=boston.feature_names)

# add variable for intercept
boston_X_df['intercept'] = 1

In [4]:
boston_mlr_sm = sm.OLS(boston_y, boston_X_df.loc[:, ['LSTAT', 'AGE', 'intercept']]).fit()

print(boston_mlr_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     309.0
Date:                Mon, 18 Apr 2022   Prob (F-statistic):           2.98e-88
Time:                        17:17:37   Log-Likelihood:                -1637.5
No. Observations:                 506   AIC:                             3281.
Df Residuals:                     503   BIC:                             3294.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
LSTAT         -1.0321      0.048    -21.416      0.0

In [5]:
boston_mlr_all_sm = sm.OLS(boston_y, boston_X_df).fit()

print(boston_mlr_all_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 18 Apr 2022   Prob (F-statistic):          6.72e-135
Time:                        17:18:55   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.1080      0.033     -3.287      0.0

In [10]:
n = len(boston_X_df)
p = len(boston_X_df.columns)
RSS = (boston_mlr_all_sm.resid**2).sum()
RSE = np.sqrt(RSS / (n - p - 1))
print(RSE)

4.750128002982112


In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.Series([
    variance_inflation_factor(boston_X_df.values, i)
    for i in range(boston_X_df.shape[1])
], index=boston_X_df.columns)

CRIM           1.792192
ZN             2.298758
INDUS          3.991596
CHAS           1.073995
NOX            4.393720
RM             1.933744
AGE            3.100826
DIS            3.955945
RAD            7.484496
TAX            9.008554
PTRATIO        1.799084
B              1.348521
LSTAT          2.941491
intercept    585.265238
dtype: float64

In [32]:
from statsmodels.tools.eval_measures import rmse
RMSE = rmse(boston_y, boston_mlr_all_sm.fittedvalues)
print(RMSE)

4.679191295697281


In [33]:
RMSE2 = np.sqrt(((boston_y - boston_mlr_all_sm.fittedvalues)**2).mean())
print(RMSE2)

4.679191295697285


In [21]:
# Add the target variable "PRICE" to the features data frame
boston_data = boston_X_df.copy()
boston_data['PRICE'] = boston_y

all_columns = ' + '.join(boston_X_df.columns)

# Fit the OLS model using R-style notation
boston_mlr = smf.ols(formula='PRICE ~ {} -AGE'.format(all_columns), data=boston_data).fit()
print(boston_mlr.summary())

                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     117.3
Date:                Mon, 18 Apr 2022   Prob (F-statistic):          6.08e-136
Time:                        18:10:38   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3024.
Df Residuals:                     493   BIC:                             3079.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     18.2185      2.540      7.172      0.0

In [22]:
boston_mlr_interact = smf.ols(formula='PRICE ~ LSTAT * AGE'.format(all_columns), data=boston_data).fit()
print(boston_mlr_interact.summary())

                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     209.3
Date:                Mon, 18 Apr 2022   Prob (F-statistic):           4.86e-88
Time:                        20:10:19   Log-Likelihood:                -1635.0
No. Observations:                 506   AIC:                             3278.
Df Residuals:                     502   BIC:                             3295.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.0885      1.470     24.553      0.0

In [51]:
def square(x):
    return x**2

boston_mlr_nonlinear = smf.ols(formula='PRICE ~ AGE**2 + AGE', data=boston_data).fit()
print(boston_mlr_nonlinear.summary())

                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     83.48
Date:                Tue, 19 Apr 2022   Prob (F-statistic):           1.57e-18
Time:                        12:30:50   Log-Likelihood:                -1801.5
No. Observations:                 506   AIC:                             3607.
Df Residuals:                     504   BIC:                             3615.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     30.9787      0.999     31.006      0.0

In [26]:
boston_mlr_nonlinear2 = smf.ols(formula='PRICE ~ np.sqrt(LSTAT) + LSTAT', data=boston_data).fit()
print(boston_mlr_nonlinear2.summary())

                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.668
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     506.4
Date:                Tue, 19 Apr 2022   Prob (F-statistic):          3.30e-121
Time:                        10:57:15   Log-Likelihood:                -1561.2
No. Observations:                 506   AIC:                             3128.
Df Residuals:                     503   BIC:                             3141.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         68.6836      2.535     27.

In [50]:
from statsmodels.stats.anova import anova_lm
m01 = smf.ols(formula='PRICE ~ AGE', data=boston_data).fit()
m02 = smf.ols(formula='PRICE ~ AGE^2 + AGE', data=boston_data).fit()
# anova_results = anova_lm(m01, m02)
# print(anova_results)
m01.summary()

PatsyError: Error evaluating factor: TypeError: Cannot perform 'xor' with a dtyped [float64] array and scalar of type [bool]
    PRICE ~ AGE^2 + AGE
            ^^^^^

In [48]:
m02.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PRICE   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     83.48
Date:                Tue, 19 Apr 2022   Prob (F-statistic):           1.57e-18
Time:                        12:30:14   Log-Likelihood:                -1801.5
No. Observations:                 506   AIC:                             3607.
Df Residuals:                     504   BIC:                             3615.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     30.9787      0.999     31.006      0.000      29.016      32.942
AGE           -0.1232      0.013     -9.137      0.000      -0.150      -0.097
==============================================================================
Omnibus:                      170.034   Durbin-Watson:                   0.613
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              456.983
Skew:                           1.671   Prob(JB):                    5.85e-100
Kurtosis:                       6.240   Cond. No.                         195.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""